## Import Libraries

In [ ]:
# Import necessary libraries for data handling, optimization, and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random, time, copy
import pickle
from itertools import permutations
from gurobipy import *
from gurobipy import GRB

In [ ]:
# Set your working directory
my_folder = "C:/Users/hyunwoolee/OneDrive - Virginia Tech/Hyunwoo Research"

## General Functions

In [ ]:
def from_x_to_obj(selected_items):
    """
    Converts a given strategy profile (selected_items) into
    binary decision variables and computes both individual (selfish)
    and global objective values.

    Parameters:
        selected_items (dict): Dictionary mapping each player to their selected items.

    Returns:
        selfish_obj (dict): Objective value for each individual player.
        global_obj (float): Total objective value across all players.
    """

    # Initialize binary decision variables x[player][item]
    x = {}
    for player in players:
        x[player] = {
            i: 1 if i in selected_items[player] else 0
            for i in items
        }

    # Compute individual (selfish) objective for each player
    selfish_obj = {}
    for player in players:
        # Direct profit + interaction-based profit
        selfish_obj[player] = sum(profits[player][i] * x[player][i] for i in items) + sum(interactions[action][i] * x[action[0]][i] * x[action[1]][i]
                for i in items for action in players_interaction if action[0] == player)

    # Compute global objective as the sum of all players' objectives
    global_obj = sum(selfish_obj[player] for player in players)

    return selfish_obj, global_obj


In [ ]:
def printSolution(model,x, selected_items):    
    
    # Check if the model found a feasible solution
    if model.SolCount > 0:
        print('\nObjvalue: %g' % model.ObjVal)
        for player in players:
            for i in items:
                if x[player][i].X > 0.1:
                    #print('item %d for player %d: %g' % (i, player, x[player][i].X))
                    selected_items[player].append(i)
    else:
        pass # No feasible solution found

## Cycle_diagnostic

In [ ]:
def Cycle_Diagnostic(selected_items):
    # Initialize flag for cycle detection
    Cycle_found = False
    S = []
    
    # Check for duplicate lake entries (indicates a cycle)
    for i in range(len(selected_items)):
        if selected_items[i] in S:
            # print('\n Cycle detected =================================================================================\n')
            Cycle_found = True
            return Cycle_found
        S.append(selected_items[i])
            
    return Cycle_found            

## Social Welfare Model

In [ ]:
### This function builds and solves a Gurobi model that maximizes social welfare

def SW_model(time_limit, usage):

    start_time = time.time()
    
    # Gurobi Model
    model = Model("SW model")
    
    # Decision variables
    x={}
    for player in players:
        x[player] = {}
        for i in items:
            x[player][i] = model.addVar(vtype = GRB.BINARY,name="x_[%s][%s]"%(str(player),str(i)))
 
    y={}
    for action in players_interaction:
        y[action] = {}
        for i in items:
            y[action][i] = model.addVar(vtype = GRB.BINARY,name="y_[%s][%s]"%(str(action),str(i)))
            
    # Player-level knapsack constraints
    for player in players:
        model.addConstr( quicksum(weights[player][i]*x[player][i] for i in items)  <= budgets[player])
        
    # Logical constraints for interaction variables y[action][i]
    for action in players_interaction:
        for i in items:
            model.addConstr(y[action][i] <= x[action[0]][i])
            model.addConstr(y[action][i] <= x[action[1]][i])
            model.addConstr(y[action][i] >= x[action[0]][i]+x[action[1]][i]-1)
    
    # === Objective function: maximize total social welfare ===
    model.setObjective(quicksum(profits[player][i]*x[player][i] for i in items for player in players) 
                       + quicksum(interactions[action][i]*y[action][i] for i in items for action in players_interaction)
                       ,GRB.MAXIMIZE)

    # Set time limit based on usage type
    if usage == 'OSW':
        model.setParam('TimeLimit',time_limit)
    elif usage == 'OSW_warm':
        model.setParam('TimeLimit',time_limit/6)

    # Solver parameters    
    model.Params.LogToConsole = 0
    model.Params.Threads =  user_Threads

    # Solve the model
    model.optimize()
    runtime=model.Runtime
    
    # Extract selected items from solution
    selected_items = {player: [] for player in players}
    printSolution(model,x, selected_items)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    return model, elapsed_time, selected_items

## Best-response problem

In [ ]:
### This function defines the best-response problem for a single player (player_fixed)
### Given other players' strategies, it solves the player's own knapsack problem.

def selfish_KP(player_fixed):
    # Initialize a Gurobi model for the player's selfish optimization
    model = Model("KP")
    model.Params.LogToConsole = 0
    model.Params.Threads = user_Threads

    # Decision variables: x[player][item] for all players (only player_fixed will be free)
    x = {
        player: {
            i: model.addVar(vtype=GRB.BINARY, name=f"x_[{player}][{i}]")
            for i in items
        }
        for player in players
    }

    # Knapsack constraint for the selected player
    model.addConstr(
        quicksum(weights[player_fixed][i] * x[player_fixed][i] for i in items) <= budgets[player_fixed]
    )

    # Objective: maximize the player's utility (direct + pairwise interaction terms)
    model.setObjective(
        quicksum(profits[player_fixed][i] * x[player_fixed][i] for i in items) +
        quicksum(interactions[action][i] * x[action[0]][i] * x[action[1]][i]
                 for action in players_interaction if action[0] == player_fixed
                 for i in items),
        GRB.MAXIMIZE
    )

    model.update()
    return model


## CR-BRD algorithm

In [ ]:
def generate_feasible_solution(budgets, items, weights):
    """
    Generates a feasible knapsack solution ensuring high budget utilization with randomness.

    Args:
        budgets (dict): Available budget for each player.
        items (list): List of all available items.
        weights (dict): Dictionary of item weights per player.

    Returns:
        dict: A dictionary where each player is assigned a feasible set of selected items.
    """
    x_current = {}

    for player in players:
        selected_items = set()
        remaining_budget = budgets[player]

        # Step 1: Shuffle items to introduce randomness
        shuffled_items = items[:]
        random.shuffle(shuffled_items)

        # Step 2: Iteratively add items while staying within the budget
        for item in shuffled_items:
            if weights[player][item] <= remaining_budget:
                selected_items.add(item)
                remaining_budget -= weights[player][item]

        x_current[player] = list(selected_items)  # Convert set to list

    return x_current

In [ ]:
def BRD(x_current, selfish_model, max_init, max_iteration):
    """
    Executes the Clockwork-Random Best-Response Dynamics (CR-BRD) algorithm for the KPG game.

    Starting from an initial strategy profile, this function iteratively updates each player's strategy
    using best-response optimization. If cycles are detected, randomization is introduced into the
    playing sequence to escape local loops. The algorithm searches for a Pure Nash Equilibrium (PNE).

    Args:
        x_current (dict): Initial strategy profile; mapping from player to list of selected items.
        selfish_model (dict or None): Dictionary of pre-built Gurobi models for each player's
                                      best-response problem. If None, models will be built on demand.
        max_init (int): Maximum number of randomized initial profiles to explore.
        max_iteration (int): Maximum number of BRD updates per initialization.

    Returns:
        tuple:
            selected_items_final (dict): Final strategy profile (player → selected items).
            BR_PNE_found (bool): True if a Pure Nash Equilibrium was found.
            Cycle_found (bool): True if a cycle was detected during any run.
            total_iterations (int): Total number of best-response updates performed.
            solution (dict): Final utility values for each player under the final profile.
    """
    
    global BR_PNE_found, I_c

    # Initialize selected_items and solution dictionary
    selected_items = {}
    selected_items[0] = {}
    for player in players:
        selected_items[0][player] = [i for i in x_current[player]]        
    solution = {}
    solution[0] = from_x_to_obj(x_current)[0]

    # Create a container for selfish models if not already provided
    if selfish_model == None:
        selfish_model = {}

    # Initial settings
    playing_sequence = players
    randomized, Cycle_found = False, False
    total_iterations = 0
    
    # Try up to max_init different initial strategy profiles
    for num_init in range(1,max_init+1):
        if num_init > 1:
            ## Generate any strategy profile
            x_current = generate_feasible_solution(budgets, items, weights)
            
            # Reset initial selections and utilities            
            selected_items = {}
            selected_items[0] = {}
            for player in players:
                selected_items[0][player] = [i for i in x_current[player]]        
            solution = {}
            solution[0] = from_x_to_obj(x_current)[0]
        
        # Run best-response updates up to max_iteration times
        for num in range(1,max_iteration+1):
            total_iterations += 1

            selected_items[num] = {}
            solution[num] = {}
            for player in players:
                solution[num][player] = 0
                selected_items[num][player] = selected_items[num-1][player]

            for player in playing_sequence:
                # Reuse or initialize best-response model for this player
                if player not in selfish_model:
                    selfish_model[player] = selfish_KP(player)

                # Fixed decisions from other players are used to set values
                for p in players_complement[player]:
                    selected_items_previous = [item for item in selected_items[num][p]]
                    for item in items:                     
                        var = selfish_model[player].getVarByName(f"x_[{p}][{item}]")
                        if item in selected_items_previous:                
                            var.lb, var.ub = 1, 1
                        else:
                            var.lb, var.ub = 0, 0
                            
                # Warm-starting with the current strategy profile
                selected_items_previous = [item for item in selected_items[num][player]]
                selfish_model[player].getVarByName(f"x_[{player}][{item}]").start = 1 if item in selected_items_previous else 0

                selfish_model[player].update()
                selfish_model[player].optimize()
                
                # Store best-response outcome
                solution[num][player] = selfish_model[player].ObjVal
                selected_items[num][player] = [item for item in items if selfish_model[player].getVarByName(f"x_[{player}][{item}]").X > 0.1]
                                    
            # Check for convergence to a Pure Nash Equilibrium
            if selected_items[len(selected_items)-1] == selected_items[len(selected_items)-2]:
                # print(' PNE found at %d iteration ================================================================== '%(num))
                BR_PNE_found = True
                selected_items_final = selected_items[num]

                return selected_items_final, BR_PNE_found, Cycle_found, total_iterations, solution[num]  
        
            # Enable random play order if a cycle is detected
            if randomized:
                players_tmp = players.copy()
                random.shuffle(players_tmp)
                playing_sequence = players_tmp
            else:
                Cycle_found = Cycle_Diagnostic(selected_items) 
                if Cycle_found:                    
                    randomized = True
                    players_tmp = players.copy()
                    random.shuffle(players_tmp)
                    playing_sequence = players_tmp

    # No equilibrium found after all initializations and iterations        
    return {player: [] for player in players}, False, True, total_iterations, {player:0 for player in players}     

## Strategic Inequalities for KPG

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    """
    Generates all subsets (power set) of a given iterable.
    Args:
        iterable (iterable): Input elements.
    Returns:
        generator: All possible subsets of the input.
    """
    
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

In [ ]:
def minimal_subsets_indices_with_negative_sum(scalar, value_list):
    """
    Finds the first subset (with smallest size) of value_list such that 
    the adjusted sum (scalar + sum(subset)) is negative.

    The function terminates as soon as it finds the first subset (in enumeration order)
    that satisfies this condition and is shorter than any previous one.

    Args:
        scalar (float): Baseline value to start the sum.
        value_list (list of float): List of values to choose from.

    Returns:
        tuple: Indices of the first minimal subset whose adjusted sum is negative.
               Returns None if no such subset exists.
    """
    min_indices = None
    min_sum = float('inf')
    min_len = float('inf')

    for indices in powerset(range(len(value_list))):
        subset_sum = scalar + sum(value_list[i] for i in indices)
        if subset_sum < 0 and len(indices) < min_len:
            min_sum = subset_sum
            min_len = len(indices)
            min_indices = indices
            # Early exit if smallest possible subset (empty) has negative sum
            if min_len == 0:
                break

    return min_indices

In [ ]:
def strategic_calculation(category):
    """
    Computes strategic inequalities for the KPG model, including:
    - Strategic Dominance Inequalities: If one item strictly dominates another.
    - Strategic Payoff Inequalities: To avoid negative profie based on the other player's choices.

    Args:
        category (str): Type of interaction (e.g., 'C' for negative interactions).

    Returns:
        tuple:
            dominance_relationships (list): List of (player, item_dominant, item_dominated) tuples.
            strategic_payoff_relationships (list): List of [player, item, subset_indices, subset_size].
    """

    # === Strategic Dominance Inequality Computation ===
    profit_interactions = {player: {} for player in players}
    profit_max = {player: {} for player in players}
    profit_min = {player: {} for player in players}
    items_pair = list(permutations(items, 2))

    for player in players:
        for item in items:
            # Interaction terms where the player is the first actor
            profit_interactions[player][item] = [
                interactions[action][item] for action in players_interaction if action[0] == player
            ]

            # Compute all possible sums with and without interaction
            min_val, max_val = float('inf'), float('-inf')
            for subset in powerset(profit_interactions[player][item]):
                total = profits[player][item] + sum(subset)
                min_val = min(min_val, total)
                max_val = max(max_val, total)

            profit_min[player][item] = min_val
            profit_max[player][item] = max_val

    # Identify item dominance relationships
    dominance_relationships = []
    for player in players:
        for i1, i2 in items_pair:
            if profit_min[player][i1] > profit_max[player][i2]:
                if weights[player][i1] <= weights[player][i2]:
                    dominance_relationships.append((player, i1, i2))

    # === Strategic Payoff Inequality Computation ===
    strategic_payoff_relationships = []

    if category == 'C':
        for player in players:
            for item in items:
                minimal_subset = minimal_subsets_indices_with_negative_sum(
                    profits[player][item], profit_interactions[player][item]
                )

                if minimal_subset:
                    strategic_payoff_relationships.append(
                        [player, item, list(minimal_subset), len(minimal_subset)]
                    )

        # Adjust interaction indices to match original interaction structure
        for str_payoff in strategic_payoff_relationships:
            pivot = str_payoff[0]
            for i in range(len(str_payoff[2])):
                if pivot <= str_payoff[2][i]:
                    str_payoff[2][i] += 1

    return dominance_relationships, strategic_payoff_relationships

## Separation Oracle

In [ ]:
### This callback function integrates a separation oracle to enforce equilibrium inequalities

def Separation(model, where):
    global x, y, EI_cuts, selfish_model, first_PNE_obj, first_time, start_time, PNE_sets

    violated = False

    # Trigger only when a new feasible integer solution is found
    if where == GRB.Callback.MIPSOL:
        # Extract current incumbent solution
        x_current = {player: [] for player in players}
        for player in players:
            x_current[player] = [i for i in items if model.cbGetSolution(x[player][i]) > 0.3 ]
        curr_selfish_obj, curr_global_obj = from_x_to_obj(x_current) 
        
        for player in players:
            # Reuse or create a selfish model for this county
            if player not in selfish_model:
                selfish_model[player] = selfish_KP(player)
                
            # Fix other counties’ decisions in the model
            for p in players_complement[player]:
                for item in items:                     
                    var = selfish_model[player].getVarByName(f"x_[{p}][{item}]")
                    if item in x_current[p]:                
                        var.lb, var.ub = 1, 1
                    else:
                        var.lb, var.ub = 0, 0
                        
            # Warm-starting with the current strategy profile
            for item in items:
                selfish_model[player].getVarByName(f"x_[{player}][{item}]").start = 1 if item in x_current[player] else 0
                    
            # Solve the player’s best-response problem
            selfish_model[player].optimize()
            
            # If a better response exists, add a lazy cut to eliminate the deviation
            if selfish_model[player].ObjVal > curr_selfish_obj[player]:
                best_response_items = [item for item in items if selfish_model[player].getVarByName(f"x_[{player}][{item}]").X > 0.3]
                # print(f'Best utility: {selfish_model[player].ObjVal}')
                # print(f'Current utility: {curr_selfish_obj[player]}')                
                # print('Best utility is higher than current utility')
                # print("\n"+"="*35 + "  Call Lazy cuts  " + "="*35)
                
                # Add equilibrium inequality (lazy cut)
                model.cbLazy(
                    sum(profits[player][i]*1 for i in items if i in best_response_items)
                    +quicksum(interactions[action][i]*1*x[action[1]][i] for i in items for action in players_interaction if (action[0]==player) and (i in best_response_items))
                    <= quicksum(profits[player][i]*x[player][i] for i in items) 
                   + quicksum(interactions[action][i]*y[action][i] for i in items for action in players_interaction if action[0]==player)
                )
                EI_cuts.append(1)
                violated = True

            # print("\n"+"="*35 + "  Continue Main program  " + "="*35)
            # print("    Nodes    |    Current Node    |     Objective Bounds      |     Work") 
            # print(" Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time")

        # Record the first PNE (Pure Nash Equilibrium) and its discovery time
        if violated == False and first_PNE_obj == None:
            first_PNE_obj = curr_global_obj 
            first_time = time.time() - start_time

        # Store the new PNE if it's unique
        if (violated == False) and (x_current not in PNE_sets):
            PNE_sets.append(x_current)


## CR-BRD Incorporated Separation Oracle

In [ ]:
### This callback function integrates both a separation oracle and the CR-BRD algorithm

def BRD_Separation(model, where):
    global x, y, EI_cuts, PNE_cuts, selfish_model, first_PNE_obj, first_time, start_time, best_obj, BR_sets, PNE_sets

    violated = False

    # Trigger only when a new feasible integer solution is found
    if where == GRB.Callback.MIPSOL:
        # Extract the current solution (selected lakes)
        x_current = {player: [] for player in players}
        for player in players:
            x_current[player] = [i for i in items if model.cbGetSolution(x[player][i]) > 0.3 ]            
        curr_selfish_obj, curr_global_obj = from_x_to_obj(x_current)    
        
        for player in players:
            # Reuse or initialize selfish model for the county
            if player not in selfish_model:
                selfish_model[player] = selfish_KP(player)
                
            # Fix other counties’ decisions in the model
            for p in players_complement[player]:
                for item in items:                     
                    var = selfish_model[player].getVarByName(f"x_[{p}][{item}]")
                    if item in x_current[p]:                
                        var.lb, var.ub = 1, 1
                    else:
                        var.lb, var.ub = 0, 0

            # Warm-starting with the current strategy profile
            for item in items:
                selfish_model[player].getVarByName(f"x_[{player}][{item}]").start = 1 if item in x_current[player] else 0
                    
            # Solve the player's best-response problem
            selfish_model[player].optimize()
            
            # If a better response exists, add a lazy cut to eliminate the deviation
            if selfish_model[player].ObjVal > curr_selfish_obj[player]:
                best_response_items = [item for item in items if selfish_model[player].getVarByName(f"x_[{player}][{item}]").X > 0.3]
                # print(f'Best utility: {selfish_model[player].ObjVal}')
                # print(f'Current utility: {curr_selfish_obj[player]}')                
                # print('Best utility is higher than current utility')
                # print("\n"+"="*35 + "  Call Lazy cuts  " + "="*35)
                
                # Add equilibrium inequality (lazy cut)
                model.cbLazy(
                    sum(profits[player][i]*1 for i in items if i in best_response_items)
                    +quicksum(interactions[action][i]*1*x[action[1]][i] for i in items for action in players_interaction if (action[0]==player) and (i in best_response_items))
                    <= quicksum(profits[player][i]*x[player][i] for i in items) 
                   + quicksum(interactions[action][i]*y[action][i] for i in items for action in players_interaction if action[0]==player)
                )
                EI_cuts.append(1)
                violated = True
                
        # Record the first PNE (Pure Nash Equilibrium) and its discovery time
        if violated == False and first_PNE_obj == None:
            first_PNE_obj = curr_global_obj 
            first_time = time.time() - start_time
            
        # print("\n"+"="*35 + "  Continue Main program  " + "="*35)
        # print("    Nodes    |    Current Node    |     Objective Bounds      |     Work") 
        # print(" Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time")

        # Apply CR-BRD if the current solution is a non-PNE
        if violated == False:
            BRD_items, BR_PNE_found, current_sol = x_current, True, curr_selfish_obj
        else:        
            BRD_items, BR_PNE_found, Cycle_found, total_iterations, current_sol  = BRD(x_current, selfish_model, max_init=5, max_iteration=20)
        
        # If a new PNE is found through separation or CR-BRD
        if (BR_PNE_found == True) and (BRD_items not in PNE_sets):
            curr_global_obj = sum(current_sol[player] for player in players)
            for player in players:
                
                # If this best response for a county hasn't been found before, add a PNE inequality
                if BRD_items[player] not in BR_sets[player]:
                    # Add PNE inequality (lazy cut)
                    model.cbLazy(
                        sum(profits[player][i]*1 for i in items if i in BRD_items[player])
                        +quicksum(interactions[action][i]*1*x[action[1]][i] for i in items for action in players_interaction if (action[0]==player) and (i in BRD_items[player]))
                        <= quicksum(profits[player][i]*x[player][i] for i in items) 
                       + quicksum(interactions[action][i]*y[action][i] for i in items for action in players_interaction if action[0]==player)
                    )
                    PNE_cuts.append(1)
                    BR_sets[player].append(BRD_items[player])
                
            # Update primal solution if this PNE improves the global objective
            if curr_global_obj > best_obj:
                best_obj = curr_global_obj
                for player in players:
                    for i in BRD_items[player]:            
                        model.cbSetSolution(x[player][i], 1)
    
            # Track this new PNE
            PNE_sets.append(BRD_items)
            
            # Store the new PNE if it's unique
            if first_PNE_obj == None:
                first_PNE_obj = curr_global_obj 
                first_time = time.time() - start_time

## ZR and BZR algorithms

In [ ]:
def ZR(time_limit, warm_start_PNE, category, callback_type):
    """
    Zero-Regret (ZR) algorithm solver for the KPG game using Gurobi.

    This function adds equilibrium inequalities through callback-based separation or CR-BRD augmentation.
    Optionally includes strategic dominance and payoff inequalities for smaller instances.

    Args:
        time_limit (float): Time budget for solving the MIP model (in seconds).
        warm_start_PNE (bool): Not used in current logic, but can enable warm-starting in the future.
        category (str): Indicates interaction type; used for conditional cut generation (e.g., 'A', 'B', or 'C').
        callback_type (str): 
            - 'Separation' applies basic equilibrium inequalities (ZR).
            - 'BRD_Separation' adds CR-BRD enhanced equilibrium cuts (BZR).

    Returns:
        tuple:
            model (gurobipy.Model): Solved Gurobi model instance.
            elapsed_time (float): Total wall-clock time to solve.
            best_response_items (dict): Selected items per player from final solution.
            first_PNE_obj (float): Objective value of the first identified PNE, if any.
            first_time (float): Wall-clock time when the first PNE was discovered.
            cuts_summary (list): [#EI_basic, #EI_dominance, #EI_payoff, #PNE_cuts].
            PNE_sets (list): List of all identified Pure Nash Equilibria.
    """
    
    global x, y, EI_cuts, PNE_cuts, selfish_model, first_PNE_obj, first_time, start_time, best_obj, BR_sets, PNE_sets
    
    start_time = time.time()

    # Initialize storage for cuts, models, and tracking info
    EI_P_cuts, EI_D_cuts, EI_cuts, PNE_cuts = [], [], [], []
    BR_sets = {player: [] for player in players}
    first_PNE_obj, first_time, best_obj = None, None, -10000

    
    # Strategic inequalities are used only for small instances (n <= 15)
    if len(players) <= 15:
        dominance_relationships, strategic_payoff_relationships = strategic_calculation(category)
    selfish_model = {}
    PNE_sets = []

    # Build Gurobi model
    model = Model(f"ZR_model")

    # Decision variables
    x={}
    for player in players:
        x[player] = {}
        for i in items:
            x[player][i] = model.addVar(vtype = GRB.BINARY,name="x_var_[%s][%s]"%(str(player),str(i)))

    y={}
    for action in players_interaction:
        y[action] = {}
        for i in items:
            y[action][i] = model.addVar(vtype = GRB.BINARY,name="y_var_[%s][%s]"%(str(player),str(i)))

    # Player-level knapsack constraints
    for player in players:
        model.addConstr( quicksum(weights[player][i]*x[player][i] for i in items)  <= budgets[player])

    # Logical constraints for interaction variables y[action][i]  
    for action in players_interaction:
        for i in items:
            model.addConstr(y[action][i] <= x[action[0]][i])
            model.addConstr(y[action][i] <= x[action[1]][i])
            model.addConstr(y[action][i] >= x[action[0]][i]+x[action[1]][i]-1)

    # === Add strategic inequalities for small-scale games ===
    if len(players) <= 15:
        # Strategic Dominance Inequalities
        for player, i1, i2 in dominance_relationships:
            model.addConstr(x[player][i2] <= x[player][i1])
            EI_D_cuts.append((player, i1, i2))
                
        # Strategic Payoff Inequalities
        for str_pay in strategic_payoff_relationships: 
            model.addConstr( x[str_pay[0]][str_pay[1]] + quicksum(x[other_player][str_pay[1]] for other_player in str_pay[2]) <= str_pay[3])
            EI_P_cuts.append(str_pay)    
            
    # === Objective: Maximize social welfare (individual + pairwise interaction) ===
    model.setObjective(quicksum(profits[player][i]*x[player][i] for i in items for player in players) 
                       + quicksum(interactions[action][i]*y[action][i] for i in items for action in players_interaction)
                       ,GRB.MAXIMIZE)

    # === Gurobi solver settings ===
    model.Params.LazyConstraints = 1    
    model.Params.Threads =  user_Threads
    model.Params.LogToConsole = 0
    
    # Log file naming based on callback type
    if callback_type == 'Separation':
        model.setParam("LogFile", f"Log_ZR_{filename}")            
    elif callback_type == 'BRD_Separation':
        model.setParam("LogFile", f"Log_BZR_{filename}")            
        
    # Update time limit to account for setup time
    remaining_time = time_limit - (time.time() - start_time)
    model.setParam('TimeLimit',remaining_time)
    
    # Launch solver with appropriate callback
    if callback_type == 'Separation':
        model.optimize(Separation)
    elif callback_type == 'BRD_Separation':
        model.optimize(BRD_Separation)

    runtime=model.Runtime
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # === Extract final solution ===
    best_response_items = {player: [] for player in players}
    printSolution(model,x, best_response_items)
    
    return model, elapsed_time, best_response_items, first_PNE_obj, first_time, [len(EI_cuts), len(EI_D_cuts), len(EI_P_cuts), len(PNE_cuts)], PNE_sets

## Experiment Loop: Run All Settings for KPG Dataset

In [ ]:
category_set = ['A', 'B', 'C']
standard_time_limit = 1800
user_Threads = 16

for category in category_set:
    directory_path = f"{my_folder}/BRP_ZR_KPG_100/KPG_generated/type_{category}"
    filenames = os.listdir(directory_path)

    for filename in filenames:
        print("=" * 100)
        print(f"Processing file: {filename}")

        # === Load Instance File ===
        with open(os.path.join(directory_path, filename), 'r') as file:
            lines = file.readlines()

        chars = filename.split('-')  # Used for filename parsing

        # === Parse Game Parameters ===
        num_players, num_items = map(int, lines[0].split())
        items = list(range(num_items))
        players = list(range(num_players))
        players_complement = {player: [p for p in players if p != player] for player in players}
        budgets = list(map(float, lines[1].split()))

        # === Initialize Profits, Weights, and Interactions ===
        profits = {player: {} for player in players}
        weights = {player: {} for player in players}
        players_interaction = list(permutations(players, 2))
        interactions = {action: {} for action in players_interaction}

        for line in lines[2:]:
            data = list(map(int, line.split()))
            item = data[0]

            for player in players:
                profits[player][item] = data[2 * player + 1]
                weights[player][item] = data[2 * player + 2]

            offset = 2 * len(players) + 1
            for idx, action in enumerate(players_interaction):
                interactions[action][item] = data[offset + idx]

        # === Run OSW (Centralized Welfare Maximization) ===
        print("=" * 35 + "  SW_model  " + "=" * 35)
        OSW, time_OSW, OSW_selected_items = SW_model(standard_time_limit, 'OSW')
        selfish_obj_OSW, global_obj_OSW = from_x_to_obj(OSW_selected_items)
        print('global obj: ',global_obj_OSW)

        # === Run BRD (initial strategy profile: 0) === #
        print("=" * 35 + "  BRD(0)_model  " + "=" * 35)
        x_current = {player: [] for player in players}
        start_time = time.time()
        first_PNE, PNE_found_BRD_0,  Cycle_found_BRD_0, iterations_BRD_0, obj  = BRD(x_current, None, max_init=10, max_iteration=20)
        end_time = time.time()
        time_BRD_0 = end_time - start_time
        selfish_obj_BRD_0, global_obj_BRD_0 = from_x_to_obj(first_PNE)
        print('global obj: ',global_obj_BRD_0)
        
        # === Run BRD (initial strategy profile: time-limited OSW solution) === #
        print("=" * 35 + "  BRD(OSW)_model  " + "=" * 35)
        OSWW, time_OSWW, OSWW_selected_items = SW_model(standard_time_limit, 'OSW_warm')
        start_time = time.time()
        near_best_PNE, PNE_found_BRD_OSW, Cycle_found_BRD_OSW, iterations_BRD_OSW, obj  = BRD(OSWW_selected_items, None, max_init=10, max_iteration=20)
        end_time = time.time()
        time_BRD_OSW = end_time - start_time + time_OSWW
        selfish_obj_BRD_OSW, global_obj_BRD_OSW = from_x_to_obj(near_best_PNE)
        print('global obj: ',global_obj_BRD_OSW)

        # === Run ZR with Basic Separation Oracle === #
        print("=" * 35 + "  ZR_model  " + "=" * 35)
        model_ZR, time_ZR, best_PNE_ZR, global_obj_first_ZR, first_time_ZR, num_cuts_ZR, PNE_sets_ZR = ZR(standard_time_limit, None, category, 'Separation')
        selfish_obj_ZR, global_obj_ZR = from_x_to_obj(best_PNE_ZR)

        # === Run ZR with CR-BRD Enhanced Separation (BZR) === #
        print("=" * 35 + "  BZR_model  " + "=" * 35)
        model_BZR, time_BZR, best_PNE_BZR, global_obj_first_BZR, first_time_BZR, num_cuts_BZR, PNE_sets_BZR = ZR(standard_time_limit, None, category, 'BRD_Separation')
        selfish_obj_BZR, global_obj_BZR = from_x_to_obj(best_PNE_BZR)

        ##################################  Summarize Results ####################################

        # Create a DataFrame with specified indexes and columns
        indexes1 = ['Total']
    
        # Create a DataFrame with specified indexes and columns
        columns = ['filename','players','items','budgets','category',
                   'BRD(0)','BRD(OSW)','ZR_1st','ZR','BZR_1st','BZR','OSW',
                   'Cuts_ZR', 'Cuts_BZR','PNE_Cuts_BZR', 
                   'BRD(0)_T','BRD(OSW)_T','ZR_1st_T','ZR_T','BZR_1st_T','BZR_T',
                   'it_BRD(0)', 'it_BRD(OSW)', 'Cycle_BRD(0)', 'Cycle_BRD(OSW)',
                   'num_PNEs_ZR','num_PNEs_BZR','bound_ZR','bound_BZR']    
    
        df_tmp_test = pd.DataFrame(index=indexes1, columns=columns)
        
        df_tmp_test.loc['Total','filename'] = filename
        df_tmp_test.loc['Total','players'] = chars[0]
        df_tmp_test.loc['Total','items'] = chars[1]
        df_tmp_test.loc['Total','budgets'] = chars[2]
        df_tmp_test.loc['Total','category'] = category
        df_tmp_test.loc['Total','ZR_1st'] = global_obj_first_ZR
        df_tmp_test.loc['Total','ZR'] = global_obj_ZR
        df_tmp_test.loc['Total','BZR_1st'] = global_obj_first_BZR
        df_tmp_test.loc['Total','BZR'] = global_obj_BZR
        df_tmp_test.loc['Total','bound_ZR'] = str(model_ZR.ObjBound)
        df_tmp_test.loc['Total','bound_BZR'] = str(model_BZR.ObjBound)
        df_tmp_test.loc['Total','OSW'] = global_obj_OSW
        df_tmp_test.loc['Total','ZR_1st_T'] = first_time_ZR
        df_tmp_test.loc['Total','ZR_T'] = time_ZR
        df_tmp_test.loc['Total','BZR_1st_T'] = first_time_BZR
        df_tmp_test.loc['Total','BZR_T'] = time_BZR
        df_tmp_test.loc['Total','Cuts_ZR'] = num_cuts_ZR[0]+num_cuts_ZR[1]+num_cuts_ZR[2]
        df_tmp_test.loc['Total','Cuts_BZR'] = num_cuts_BZR[0]+num_cuts_BZR[1]+num_cuts_BZR[2]
        df_tmp_test.loc['Total','PNE_Cuts_BZR'] = num_cuts_BZR[3]
        df_tmp_test.loc['Total','num_PNEs_ZR'] = len(PNE_sets_ZR)
        df_tmp_test.loc['Total','num_PNEs_BZR'] = len(PNE_sets_BZR)
        df_tmp_test.loc['Total','Status_ZR'] = str(model_ZR.Status)
        df_tmp_test.loc['Total','Status_BZR'] = str(model_BZR.Status)
        
        df_tmp_test.loc['Total','BRD(0)'] = global_obj_BRD_0
        df_tmp_test.loc['Total','BRD(OSW)'] = global_obj_BRD_OSW
        df_tmp_test.loc['Total','BRD(0)_T'] = time_BRD_0
        df_tmp_test.loc['Total','BRD(OSW)_T'] = time_BRD_OSW
        df_tmp_test.loc['Total','it_BRD(0)'] = iterations_BRD_0
        df_tmp_test.loc['Total','it_BRD(OSW)'] = iterations_BRD_OSW
        df_tmp_test.loc['Total','Cycle_BRD(0)'] = Cycle_found_BRD_0
        df_tmp_test.loc['Total','Cycle_BRD(OSW)'] = Cycle_found_BRD_OSW
                     
        # === Save Results to CSV === #
        results_path = f"{my_folder}/BZR_KPG/KPG_results/KPG_test_{category}.csv"
        try:
            df_test = pd.read_csv(results_path, index_col=0)
            df_test = pd.concat([df_test, df_tmp_test])
        except FileNotFoundError:
            df_test = df_tmp_test
    
        df_test.to_csv(results_path)
